# 03. 협업 필터링 추천 시스템

- **추천 시스템의 성능응 높이기 위해, 사용자 집단(성별, 직업 등)으로 나누지 않고, 취향을 기준으로 분류**
    
    $\rightarrow$ 어떤 아이템에 대해서 비슷한 취향을 가진 사람들은 다른 아이템에 대해서도 비슷한 취향을 가지고 있을 것이라고 가정할 수 있기 때문

<br>

## 03.01. 협업 필터링의 원리
- **협업 필터링은 기본적으로 취향이 비슷한 사람들의 집단이 존재한다고 가정**
- **추천의 대상이 되는 어떤 한 사람이 있으면, 이 사람과 취향이 비슷한 사람들(neighbor)을 찾아내기만 하면 이 사람들이 공통적으로 좋아하는 제품/서비스를 추천 대상인 사람에게 추천하면 된다는 것이 기본 아이디어**

<br>

### 협업 필터링의 절차

- 5개의 영화 ($M_1 \sim M_5$)에 대한 5명의 사용자 ($U_1 \sim U_5$)의 평가에 대한 자료

<br>

| Users/Movies | $M_1$ | $M_2$ | $M_3$ | $M_4$ | $M_5$ | Correlation with $U_1$ |
| - | - | - | - | - | - | - |
| $U_1$ | $2$ | $5$ | $3$ | | | - |
| $U_2$ | $4$ | $4$ | $3$ | $5$| $1$| 0.19 |
| $U_3$ | $1$ | $5$ | $4$ | | $5$| 0.89 |
| $U_4$ | $3$ | $5$ | $3$ | $2$| $5$| 0.94 |
| $U_5$ | $4$ | $5$ | $3$ | $4$ | | 0.64 |
| Average of $U_3$ and $U_4$ | $2$ | $5$ |$3.5$ | $2$ | $5$ | |

#### 1. **$U_1$과 취향이 비슷한 사용자를 탐색**
- 취향은 영화에 대한 평가로 표현 $\rightarrow$ 각 사용자 평가의 유사성을 계산
    
    $\rightarrow$ **$U_3$과 $U_4$가 $U_1$과 가장 높은 상관관계를 보이므로, 이 둘을 취향이 비슷한 사용자 그룹인 neighbor로 분류**

#### 2. **$U_3$과 $U_4$가 가장 좋게 평가한 영화를 탐색**
- $U_1$이 아직 보지 않은 영화로는 $M_4$와 $M_5$가 존재

    $\rightarrow$ $M_4$와 $M_5$에 대한 $U_3$과 $U_4$의 평점 평균은 각각 2와 5

#### 3. $M_5$를 $U_1$에게 추천 영화로 제시

<br>

## 03.02. 유사도지표
- **CF는 사용자들의 평가를 기반으로, 사용자간 유사도를 구하는 것이 핵심**

<br>

### 상관계수 (Correlation)
- 평가 자료가 연속값인 경우에 가장 이해하기 쉬운 유사도로는 상관계수가 존재
- **$x$, $y$는 유사도의 계산 대상이 디ㅗ는 두 사용자이고,** 
    
    **$r_{x, y}, r_{y,i}$는 두 사용자가 공통으로 평가한 아이템 $I_{x,y}$중에서 $i$번째 아이템에 대한 이 두사용자의 평가 값**

$$simil(x, y) = \frac{\sum_{i \in I_{xy}} (r_{x,i}-\bar{r_x})(r_x{y, i} - \bar{r}_y)} {\sqrt{\sum_{i \in I_{x,y}}(r_{x, i} - \bar{x}_i)^2 \sum_{i \in I_{x, y}}(r_{y, i} - \bar{r}_y)^2} }$$

- 상관계수는 이해하기 쉬운 유사도 측정치이지만, 협업 필터링에서는 늘 좋은 결과를 가져오지는 못함

<br>

### 코사인 유사도 (Cosine Sim)
- 협업 필터링에서 연속값에 대해서 가장 널리 쓰이는 측정치는 코사인 유사도
- 코사인 유사도에서는 각 아이템을 하나의 차원으로 보고 사용자의 평가값을 좌표값으로 취급

    $\rightarrow$ 각 사용자의 평가값을 벡터로 해서 두 사용자 간의 벡터의 각도(코사인값)을 구할 수 있음
    
<br>

<img src = 'https://seongkyun.github.io/assets/post_img/study/2019-04-01-cosine_distance/fig3.png'>

- 두 아이템 ($A$, $B$)에 대해서 평가한 두 사용자($x$, $y$)를 표시

    **각 사용자의 평가값을 좌표로 사용해서 원점에서 사용자까지의 선을 그으면, 사용자의 벡터**
    
    **두 사용자의 평가값이 유사할수록 각도 $\theta$가 작다는 (코사인값이 크다는)것을 알 수 있음**

<br>

#### 코사인 유사도의 계산식
- $x, y$는 유사도 계산 대상이 되는 두 사용자
- $r_{x, i}, r_{y, i}$는 두 사용자가 공통적으로 평가한 아이템 $I_{x,y}$중에서 $i$번째 아이템에 대한 평가값
- 코사인값은 최소 -1 (완전 불일치)에서 최대 1 (완전 일치)까지의 값을 가짐

$$simil(x, y) = cos(\vec{x}, \vec{y})=\frac{{\vec{x} \cdot \vec{y}}}{|| \vec{x}|| \times || \vec{y}||} = \frac{\sum_{i \in I_{xy}} r_{x, i} r_{y, i}}{\sqrt{\sum_{i \in I_x} r^2_{x, i}} \sqrt{\sum_{i \in I_y} r^2_{y, i}}}$$

<br>

#### 타니모토 계수 (Tanimoto Coef)
* **만일 데이터가 이진값을 갖는다면 위의 상관계수나 코사인 유사도는 사용할 수 없음**
- 타니모토 계수는 최소 0 (완전 불일치), 최대 1 (완전 일치) 까지의 값을 가짐
- 이진수 데이터에 대해서는 협업 필터링에서 상당히 좋은 결과를 보임
- 타니모토 계수의 변형으로 자카드 계수 (Jaccard Coef) 존재


$$simil(x, y) = \frac{c}{a+b-c}$$

- $a$ : 사용자 $x$가 1의 값을 갖는 (구입한 혹은 클릭한) 아이템의 수
- $b$ : 사용자 $y$가 1의 값을 갖는 (구입한 혹은 클릭한) 아이템의 수
- $c$ : 사용자 $x$와 $y$가 공통적으로 1의 값을 갖는 아이템 수

<br>

## 03-03. 기본 CF 알고리즘
1. **모든 사용자 간의 평가 유사도를 계산**

2. **현재 추천 대상이 되는 사람과 다른 사용자의 유사도를 추출**

3. **현재 사용자가 평가하지 않은 모든 아이템에 대해서, 현재 사용자의 예상 평가 값을 구함**
- **예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를, 현재 사용자와 그 사용자와의 유사도로 가중해서 평균을 계산**

4. **아이템 중에서 예상 평가값이 가장 높은 N개의 아이템을 추천**

<br>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

- 데이터 로드

In [2]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

In [3]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies[['movie_id', 'title']]

In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings = ratings.drop('timestamp', axis=1)

- 훈련/테스트 데이터 분리

In [5]:
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

- RMSE 함수 정의

In [6]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [7]:
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

- Full Matrix 계산

In [8]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

- 모든 가능한 사용자 pair의 코사인 유사도 계산

In [9]:
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [10]:
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.106796,0.041595,0.049991,0.302451,0.311584,0.322208,0.200532,0.048157,0.262415,...,0.260261,0.115926,0.235330,0.164176,0.156255,0.111639,0.237398,0.079785,0.137199,0.315514
2,0.106796,1.000000,0.087066,0.188767,0.068244,0.188542,0.080138,0.066676,0.098563,0.104149,...,0.133790,0.312993,0.301453,0.426051,0.259404,0.243597,0.223850,0.118296,0.066183,0.115883
3,0.041595,0.087066,1.000000,0.310098,0.026668,0.044191,0.040984,0.034642,0.079659,0.054869,...,0.031096,0.053597,0.121982,0.056152,0.140056,0.014390,0.151823,0.047134,0.049532,0.033503
4,0.049991,0.188767,0.310098,1.000000,0.041919,0.067958,0.090338,0.033381,0.077101,0.064790,...,0.054176,0.048418,0.115213,0.179845,0.122496,0.038997,0.179104,0.076643,0.073220,0.062261
5,0.302451,0.068244,0.026668,0.041919,1.000000,0.225746,0.286069,0.166538,0.031562,0.180440,...,0.272904,0.078042,0.090441,0.084947,0.160676,0.091792,0.170910,0.112751,0.118227,0.211064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.111639,0.243597,0.014390,0.038997,0.091792,0.083051,0.037311,0.064803,0.053219,0.065983,...,0.068142,0.392690,0.181645,0.145146,0.331068,1.000000,0.083532,0.156504,0.039309,0.122659
940,0.237398,0.223850,0.151823,0.179104,0.170910,0.255814,0.247007,0.214504,0.063418,0.197781,...,0.182232,0.117539,0.126384,0.165944,0.195127,0.083532,1.000000,0.094852,0.152422,0.228938
941,0.079785,0.118296,0.047134,0.076643,0.112751,0.102014,0.030135,0.037736,0.142944,0.057635,...,0.035386,0.098523,0.232371,0.140754,0.181389,0.156504,0.094852,1.000000,0.036788,0.058654


- 주어진 영화 가중평균 평점을 계산
    - 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity) 

In [11]:
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        
        # 현재 사용자와 다른 사용자 간의 similarity
        sim_scores = user_similarity[user_id].copy()
        
        # 현재 영화에 대한 모든 사용자의 rating값
        movie_ratings = rating_matrix[movie_id].copy()
        
        # 현재 영화를 평가하지 않은 사용자의 index
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        
        # 현재 영화를 평가한 모든 사용자의 가중평균값
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

In [12]:
score(CF_simple)

1.014049500813803

<br>

## 03.04. 이웃을 고려한 CF
- **CF를 개선할 수 있는 한가지 방법은, 사용자 중에서 유사도가 높은 사용자를 선정해서 그 사람들의 평점만 가지고 예측을 하는 것**
    
    **이웃(neighbor)를 전체 사용자로 하는 대신에 유사도가 높은 사람만을 이웃으로 선정해서 이웃의 크기를 줄이는 것**
    
<br>

#### 이웃을 정하는 기준
1. 이웃의 크기를 미리 정해놓고 추천 대상 사요자와 가장 유사한 K명을 선택하는 KNN
2. 유사도의 기준 (예: 상관계수 0.8)이상을 정해 놓고 이 기준을 충족시키는 사용자를 이웃으로 정하는 Thresholding
- 일반적으로KNN보다 정확하기는 하지만, 정해진 기준을 넘는 사용자가 없어서 추천을 하지 못하는 경우 발생 가능

<br>



#### 이웃을 고려한 CF 구현

In [13]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [14]:
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

- Full Matrix 계산

In [15]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

- 사용자들의 코사인 유사도 계산

In [16]:
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

<br>

- Neighbor Size를 정해서 예측
    - Neighbor Size가 지정되지 않은 경우, 영화를 평가한 모든 사용자의 가중평균값 계산
    - Neighbor Size가 지정되었으나, 최소 2명이 되는 경우에만 Neighbor Size값을 사용

In [17]:
def cf_knn(user_id, movie_id, neighbor_size=0):
    
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
        if neighbor_size == 0:          
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

        else:                       
            if len(sim_scores) > 1: 
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

In [18]:
score(cf_knn, neighbor_size=30)

1.0059487828688982

<br>

#### 주어진 사용자에 대해 추천
- 전체 데이터로 Full Matrix와 코사인 유사도 계산

In [19]:
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [20]:
def recommender(user, n_items=10, neighbor_size=20):

    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index  
    items = rating_matrix.loc[user].drop(rated_index)
    
    for item in items.index:
        predictions.append(cf_knn(user, item, neighbor_size))              
        
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]   
    recommended_items = movies.loc[recommendations.index]['title']
    
    return recommended_items

- 2번 사용자와 가장 유사한 30명의 사용자를 통하여, 5개의 추천

In [21]:
recommender(user=2, n_items=5, neighbor_size=30)

movie_id
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1189                              That Old Feeling (1997)
1293                     Ayn Rand: A Sense of Life (1997)
1467                                     Cure, The (1995)
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

<br>

## 03-05. 최적의 Neighbor Size
- **이웃 크기가 지나치게 커지면, 개인의 취향이 반영되지 않음**

In [22]:
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [23]:
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0241
Neighbor size = 20 : RMSE = 1.0087
Neighbor size = 30 : RMSE = 1.0059
Neighbor size = 40 : RMSE = 1.0056
Neighbor size = 50 : RMSE = 1.0055
Neighbor size = 60 : RMSE = 1.0064


<br>

## 03.06. 사용자의 평가경향을 고려한 CF
- **사용자의 평가경향 (user bias)을 고려하여 예측치를 조정**
- 사용자에 따라서 평가를 전체적으로 높게 하는 사람이 있는 반면,
    
    평가를 전체적으로 낮게 하는 사람이 있는 등 사용자에 따라 평가경향이 상이

<br>

#### 사용자의 평가경향을 고려하지 않는 예측치

$$p_{a, i} = \frac{\sum^n_{u=1} w_{a, u} r_{u, i}}{\sum^n_{u=1} w_{a, u}}$$

- $a$ : 사용자, $u$ : 이웃 이용자, $n$ : 이웃 사용자의 수
- $p_{a,i}$ : 아이템 $i$에 대한 사용자 $a$의 예상 평점
- $w_{a, u}$ : 사용자 $a$와 $u$의 유사도
- $r_{u, i}$ : 아이템 $i$에 대한 사용자 $u$의 평점

- 사용자의 평가경향을 고려해서 예측치를 조정한다면, 정확도의 개선 가능
    - A라는 사용자는 평점평균이 2.0이고, X라는 영화에 대해서 3.0의 평점을 주었고,
        
        B라는 사용자는 평점평균이 4.0인데 같은 X영화에 똑같이 3.0의 평점을 주었다고 가정할 때,
        
        **$\rightarrow$** 같은 3.0의 평점이지만, 두 사용자의 평가경향을 고려해 보면 매우 다른 의미일 수 있음**
        
        **$\rightarrow$** **A사용자의 3.0은 높은 점수로, B사용자의 3.0은 낮은 점수로 활용되어야 더 정확한 예측이 될 수 있음**
        
    <br>
    
- **추천의 대상이 되는 사용자의 경우에도, 그 사용자의 평가경향을 고려한 조정이 필요**
    - 현재 추천의 대상이 되는 사용자의 평점평균이 3.0인 데 비해, 예측치를 구하는 데 사용된 사용자(이웃)들의 평점평균은 4.0이라고 할 때,
    
        $\rightarrow$ 만일 이웃의 원래 평점으로 계산한 예측값이 3.5라면, 현재 사용자의 평가경향을 고려하면,
            
        3.5 - 1(이웃과 현 사용자의 평점평균 차이) = 2.5가 합리적인 예측치
        
<br>
    
#### 사용자의 평가경향을 고려하는 경우의 예측치

$$p_{a, i} = \bar{r_a} + \frac{\sum^n_{u=1}w_{a, u}(r_{u, i}-\bar{r_u})}{\sum^n_{u=1} w_{a, u}}$$

- $a$ : 사용자, $u$ : 이웃 사용자, $n$ : 이웃 사용자의 수
- $p_{a, i}$ : 아이템 $i$에 대한 사용자 $a$의 예상 평점
- $w_{a, u}$ : 사용자 $a$와 $u$의 유사도
- $r_{a, i}$ : 아이템 $i$에 대한 사용자 $u$의 평점
- $\bar{r_a}$ : 사용자 $a$의 전체 평점평균
- $\bar{r_u}$ : 사용자 $u$의 전체 평점평균

<br>

- **사용자의 평가경향을 고려하는 경우의 예측**
1. 각 사용자의 평점평균을 계산

In [38]:
rating_mean = rating_matrix.mean(axis=1)
rating_mean.head()

user_id
1    3.578431
2    3.673913
3    3.000000
4    4.333333
5    2.916031
dtype: float64

2. **각 아이템의 평점을 각 사용자의 평균에서의 차이 (평점-해당 사용자의 평점 평균)으로 변환 (=평점편차)**
- **훈련 데이터의 사용자별 각 아이템의 평점과, 해당 사용자의 평점평균의 차이 계산**

In [41]:
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1669,1671,1672,1673,1674,1675,1676,1679,1680,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.421569,-0.578431,NaN,-0.578431,-0.578431,1.421569,0.421569,-2.578431,1.421569,-0.578431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.673913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0.083969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3. 평점편차를 사용해서 해당 사용자의 해당 아이템의 편차 예측값 (평점편차의 예측값)을 계산
    
    (해당 사용자의 이웃을 구하고, 이들 이웃의 해당 아이템에 대한 평점편차와 유사도를 가중평균)
4. 계산한 편차 예측값을 평균에서의 차이를 의미하기 때문에 실제 예측값으로 변환하기 위해서, 현 사용자의 평균에 이 편차 예측값을 더함
5. 예측값을 구할 수 없는 경우에 지금까지는 3.0을 할당하였지만, 해당 사용자의 평점평균으로 대체

In [34]:
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:

        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()
        
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
        # Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
        
        # Neighbor size가 지정된 경우            
        else:
            
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산            
            if len(sim_scores) > 1:
                
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

In [35]:
score(CF_knn_bias, 30)

0.9389746065899419

<br>

#### 입력된 사용자를 위해 10개의 영화를 추천

In [44]:
def recommender(user, n_items=10):

    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    # 사용자가 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    
    # 예상평점 계산
    for item in items.index:
        predictions.append(CF_knn_bias(user, item))                              
    
    # 예상평점이 가장 높은 영화 
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    
    recommended_items = movies.loc[recommendations.index]['title']
    
    return recommended_items

In [45]:
recommender(2, 10)

movie_id
814                                   One Fine Day (1996)
1536                                          Cosi (1996)
1491                               Window to Paris (1994)
1629    Silence of the Palace, The (Saimt el Qusur) (1...
1653                         Chairman of the Board (1998)
1642                                    Angel Baby (1995)
1599                                  Guantanamera (1994)
1293                     Ayn Rand: A Sense of Life (1997)
1467                                     Cure, The (1995)
1398                         Stranger in the House (1997)
Name: title, dtype: object

<br>

## 03.07. CF 정확도 개선

<br>

### 신뢰도 가중 (significance weighting)
- 사용자 A에 대한 추천을 하는 경우:
    
    $\rightarrow$ 사용자 A와 다른 사용자 간의 유사도를 계산 (사용자 A와 다른 사용자가 공통으로 평가한 아이템에 대한 평점을 사용해서 계산)
    
    $\rightarrow$ **모든 사용자별로, 사용자 A와 공통으로 평가한 아이템의 수가 각각 다를 수 있음**
    
    $\rightarrow$ **사용자 A와 특정 사용자 간의 유사도가 비슷하더라도, 이 유사도의 신뢰도는 공통으로 평가한 아이템의 수에 따라 달라짐**
    
    $\rightarrow$ **유사도를 신뢰도에 따라서 가중치 부여 (= 공통 아이템이 적은 사용자와의  유사도보다, 많은 사용자와의 유사도에 더 큰 가중치 부여)**
    
    $\rightarrow$ **단, RMSE의 변동을 방지하기 위하여, 특정 신뢰도 기준치 이상인 사용자만을, A사용자의 추천을 위해서 사용**
        
    **[신뢰도 (공통으로 평가한 아이템의 수)가 일정값 이상인 사용자만 이웃 사용자로 활용]**

- 아이템에 대한 신뢰도 가중:
    - **특정 기준치 이상인 평가수를 가진 아이템만을, 예측에 활용**

<br>

#### 1. 사용자별 공통 평가 수 계산
- 전체 rating_matrix 중, 평점이 존재하는 경우만 1로 표시 후 내적 계산
    
    $\rightarrow$ **행렬의 각 원소는 각각의 사용자가 공통으로 평가한 영화의 수, 대각선은 각 사용자가 평가한 영화의 수**

In [56]:
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)

In [57]:
counts

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,204.0,9.0,6.0,4.0,51.0,51.0,83.0,16.0,2.0,41.0,...,42.0,10.0,30.0,11.0,22.0,8.0,30.0,4.0,16.0,50.0
2,9.0,46.0,6.0,5.0,3.0,19.0,10.0,3.0,3.0,7.0,...,9.0,10.0,20.0,15.0,15.0,9.0,13.0,4.0,4.0,7.0
3,6.0,6.0,40.0,9.0,1.0,5.0,6.0,2.0,2.0,6.0,...,2.0,2.0,9.0,3.0,8.0,1.0,11.0,2.0,3.0,2.0
4,4.0,5.0,9.0,18.0,2.0,4.0,8.0,2.0,1.0,3.0,...,2.0,1.0,5.0,3.0,4.0,1.0,5.0,2.0,3.0,3.0
5,51.0,3.0,1.0,2.0,131.0,30.0,58.0,13.0,1.0,23.0,...,35.0,4.0,11.0,3.0,15.0,7.0,16.0,4.0,11.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,8.0,9.0,1.0,1.0,7.0,7.0,4.0,2.0,1.0,4.0,...,6.0,13.0,11.0,5.0,16.0,37.0,4.0,4.0,2.0,8.0
940,30.0,13.0,11.0,5.0,16.0,31.0,38.0,11.0,2.0,23.0,...,19.0,6.0,14.0,9.0,14.0,4.0,80.0,3.0,10.0,19.0
941,4.0,4.0,2.0,2.0,4.0,6.0,2.0,2.0,2.0,3.0,...,2.0,2.0,9.0,3.0,6.0,4.0,3.0,16.0,1.0,2.0


<br>

2. **입력된 사용자와 공통으로 평가한 영화의 수 (최소 신뢰도)가 `SIG_LEVEL`이상이고,** 
    
    **입력된 영화에 대한 평가의 수 (최소 사용자 수)가 `MIN_RATINGS` 이상인 데이터를 사용하여 추천**

In [68]:
def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]
        
        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]
        
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL
        
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
        # Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
        
        # Neighbor size가 지정된 경우            
        else:
            
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
        
    prediction = np.where(prediction <= 1.0, 1.0, prediction)
    prediction = np.where(prediction >= 5.0, 5.0, prediction)

    return prediction

In [69]:
SIG_LEVEL = 3
MIN_RATINGS = 2
score(CF_knn_bias_sig, 30)

0.9377293881749231

<br>

### 예측값에 대한 수정
- 영화 평가 데이터는 1 ~ 5 사이의 값으로 평가된 데이터
- CF알고리즘을 사용해서 예측을 할 시, 경우에 따라서 예측값이 1보다 크거나 5보다 큰 경우가 존재하여 RMSE를 높이는 요인으로 작용

    $\rightarrow$ 1보다 작은 값은 1로, 5보다 큰 값은 5로 수정

```python
    prediction = np.where(prediction <= 1.0, 1.0, prediction)
    prediction = np.where(prediction >= 5.0, 5.0, prediction)
    prediction = np.round(prediction)

    return prediction


```

<br>

#### 입력된 사용자를 위해 10개의 영화를 추천

In [62]:
def recommender(user, n_items=10, neighbor_size=20):
    
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    
    for item in items.index:
        predictions.append(CF_knn_bias_sig(user, item, neighbor_size))       # 예상평점 계산
        
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    
    return recommended_items

In [63]:
recommender(2, 10, 30)

movie_id
1293      Ayn Rand: A Sense of Life (1997)
1449                Golden Earrings (1947)
1175            Welcome To Sarajevo (1997)
64      What's Eating Gilbert Grape (1993)
119                      Striptease (1996)
867                Hearts and Minds (1996)
1612                     Tokyo Fist (1995)
318        Everyone Says I Love You (1996)
515                      Local Hero (1983)
302                     Ulee's Gold (1997)
Name: title, dtype: object

<br>

## 03.08. 사용자 기반 CF & 아이템 기반 CF

<br>

#### **사용자 기반 CF (User Based CF, UBCF) : 유사도를 계산하는 기준이 사용자**
- 취향이 비슷한 이웃 사용자 (neighbor)를 알아내고, 이 그룹에 속한 사용자들이 공통적으로 좋게 평가한 아이템을 추천

<br>

#### **아이템 기반 CF (Item Based CF, IBCF) : 유사도를 계산하는 기준이 아이템**
- 사용자의 평가 패턴을 바탕으로, 아이템간의 유사도를 계산해서 사용자의 특정 아이템에 대한 예측 평점을 계산
- **예측 대상 사용자가 평가한 아이템의 평점과, 다른 각 아이템과의 유사도를 가중해서 평균한 값을, 그 아이템에 대한 예측값으로 사용**

<br>

#### UBCF와 IBCF 장단점
- UBCF는 각 사용자별로 맞춤형 추천을 하기 때문에, 데이터가 풍부한 경우 정확한 추천이 가능
    
    하지만, UBCF는 정확할 때는 매우 정확하지만 터무니없는 추천을 하는 경우도 상당히 있음

- IBCF의 장점은 정확도는 떨어지지만 사용자별로 따로따로 계산을 하지 않기 때문에 계산이 빠르다는 장점
- UBCF는 데이터가 조금 바뀔 때마다 모델의 업데이트를 해야 하지만, 
    
    IBCF는 데이터가 조금 바뀌어도 추천 결과에는 영향이 크지 않기에 모델의 업데이트를 자주 하지 않아도 됨

<br>

- **데이터 크기가 적고 각 사용자에 대한 충분한 정보(구매나 평가)가 있는 경우에는 UBCF,** 
    
    **데이터가 크거나 각 사용자에 대한 충분한 정보가 없는 경우에는 IBCF가 적절**
    
- 둘 중 어느 쪽이 정확한지는 분야마다 다르지만, **데이터가 충분하다면 UBCF가 IBCF보다 다소 정확하다고 알려져 있음**
    
    **대신, IBCF가 계산이 빠르기 때문에 아마존과 같은 대규모 데이터를 다뤄야 하는 상업용 사이트에서는 IBCF에 기반한 알고리즘이 사용**

<br>

#### IBCF의 구현
- train set의 모든 가능한 아이템 pair의 Cosine similarities 계산

In [77]:
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [80]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)

item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

In [81]:
item_similarity

movie_id,1,2,3,4,5,6,7,8,9,10,...,1669,1671,1672,1673,1674,1675,1676,1679,1680,1682
movie_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.305247,0.229683,0.358781,0.221418,0.094555,0.439287,0.335137,0.397244,0.213968,...,0.0,0.0,0.038925,0.041286,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.305247,1.000000,0.226809,0.351505,0.295569,0.094144,0.307615,0.238980,0.185849,0.106334,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.095059
3,0.229683,0.226809,1.000000,0.226007,0.128728,0.082645,0.272890,0.127335,0.202867,0.119894,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.111264
4,0.358781,0.351505,0.226007,1.000000,0.293245,0.067736,0.344764,0.382435,0.330892,0.196201,...,0.0,0.0,0.045517,0.000000,0.0,0.0,0.0,0.0,0.0,0.085828
5,0.221418,0.295569,0.128728,0.293245,1.000000,0.051473,0.256585,0.207565,0.283541,0.052317,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.108465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075045,0.065902,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.000000
1676,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075045,0.065902,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.000000


<br>

- 주어진 영화의 (movie_id) 가중평균 rating을 계산
- 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)

In [75]:
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:     
        
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        
        # 사용자가 평가하지 않은 영화 제거
        non_rating_idx = user_rating[user_rating.isnull()].index
        user_rating = user_rating.dropna()
        
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

In [76]:
score(CF_IBCF)

1.0120153127525295

<br>

## 03.09. 추천 시스템의 성과측정지표

<br>

### 연속형 지표

#### MAD (Mean Absolute Deviation)

$$MAD = \frac{\sum^n_{i=1} \sum^m_{j=1} (|r_{i, j}-p_{i, j}|)\;\text{ if }\;r_{i, j} \neq null,\;0\;\;\text{otherwise}}{k}$$

#### MSE (Mean Square Error)

$$MSE = \frac{\sum^n_{i=1} \sum^m_{j=1} ((r_{i, j}-p_{i, j})^2)\;\text{ if }\;r_{i, j} \neq null,\;0\;\;\text{otherwise}}{k}$$


#### RMSE (Root Mean Squared Error)

$$RMSE=\sqrt{MSE}$$

<br>

- $n$ : train set에 있는 아이템 수
- $m$ : test set에 있는 아이템 수
- $r_{i,j}$ : 사용자 $i$의 아이템 $j$에 대한 실제 평점
- $p_{i, j}$ : 사용자 $i$의 아이템 $j$에 대한 예상 평점
- $k$ : 정확도 계산에 포함된 총 아이템 수 (null이 아닌 평점의 개수)

<br>

### 이진범주 지표

#### 정확도 (Accuracy)

$$\frac{\text{올바르게 예측된 아이템의 수}}{\text{전체 아이템의 수}}$$

#### 정밀도 (Precision)

$$\frac{\text{올바르게 추천된 아이템의 수}}{\text{전체 추천된 아이템의 수}}$$

#### 재현율 (Recall)

$$\frac{\text{올바르게 추천된 아이템의 수}}{\text{사용자가 실제 선택한 전체 아이템 수}}$$

#### 조화평균 (F1 Score)

$$\frac{\text{2 X 정밀도 X 재현율}}{\text{정밀도 + 재현율}}$$

#### 범위 (Coverage) 

$$\frac{\text{추천이 가능한 사용자 수 (혹은 아이템 수)}}{\text{전체 아이템의 수 (혹은 아이템 수)}}$$

<img src='https://i.namu.wiki/i/eSMR6d76wWErA7K-y3YfnjJVQwxL6KRi09OLDdDs41s8V7EduWXD79wTg4EFkaK1n8J4UfpUcwxW6CyzMtbpTg.webp'>